In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **FIRST BEST BLEND**

In [2]:
import pandas as pd

# this submission is the result from LGBM_AHMED 
sub_lgbm_ahmed = pd.read_csv('/content/drive/My Drive/AKEED_SOLUTION_Predictions/Akeed_LGB_ALL_DATA_Ahmed_Binary.csv')

# this submission is the result from LGBM_AZER 
sub_best_azer_lr_003 = pd.read_csv('/content/drive/My Drive/AKEED_SOLUTION_Predictions/LGBM_without_categ_features_STABLE_BINARY.csv')

# this submission is the result from LGBM_NACIR 
sub_best_nacir = pd.read_csv('/content/drive/My Drive/AKEED_SOLUTION_Predictions/AKEED_LGBM_NACIR_BINARY.csv')

# this submission is the result from LGBM_Categorical 
sub_best_azer_categorical = pd.read_csv('/content/drive/My Drive/AKEED_SOLUTION_Predictions/LGBM_CATEGORICAL_features_BINARY.csv')

In [3]:
import numpy as np

print('THe mean of each sub is : \n')
print('sub_lgbm_ahmed',np.mean(sub_lgbm_ahmed['target']),'\n')
print('sub_best_azer_lr_003',np.mean(sub_best_azer_lr_003['target']),'\n')
print( 'sub_best_nacir',np.mean(sub_best_nacir['target']),'\n')
print('sub_best_azer_categorical',np.mean(sub_best_azer_categorical['target']),'\n')

THe mean of each sub is : 

sub_lgbm_ahmed 0.019050837320574162 

sub_best_azer_lr_003 0.017504186602870814 

sub_best_nacir 0.022684808612440193 

sub_best_azer_categorical 0.017988038277511963 



In [4]:
def binary_blend(sub1,sub2,sub3,sub4) :
  final_blend = []
  for a,b,c,d in zip(list(map(bool,sub1)),list(map(bool,sub2)),
                   list(map(bool,sub3)),list(map(bool,sub4))) :
    final_blend.append(int(bool(a*b + a*c + a*d + b*c + b*d + c*d)))
    

  return final_blend

In [5]:
sub = sub_best_nacir.copy()
sub['target']= binary_blend(list(sub_best_azer_categorical['target'].values),
                            list(sub_lgbm_ahmed['target'].values),
                            list(sub_best_nacir['target'].values),
                            list(sub_best_azer_lr_003['target'].values))

In [6]:
sub['target'].value_counts()

0    1640012
1      31988
Name: target, dtype: int64

In [ ]:
sub.to_csv('LGBM_CATEG_LGBM_AHMED_LGBM_NACIR_LGBM_AZER_003.csv',index=False)

In [7]:
import numpy as np

np.mean(sub['target'])

0.01913157894736842

# **SECOND BEST BLEND**

In [ ]:
import pandas as pd 

In [ ]:
lgb_nacir = pd.read_csv('AKEED_LGBM_NACIR_FLOATS.csv')
lgb_ahmed = pd.read_csv('Akeed_LGB_ALL_DATA_Ahmed_Float_Preds.csv')
lgb_azer = pd.read_csv('LGBM_without_categ_features_STABLE_FLOATS.csv')
xgb_azer = pd.read_csv('XGBOOST_FLOATS.csv')

In [ ]:
lgb_ahmed.sort_values(by='CID X LOC_NUM X VENDOR',inplace=True)
lgb_ahmed.reset_index(drop=True,inplace=True)
lgb_ahmed.head(1)

,CID X LOC_NUM X VENDOR,target
0,000IPH5 X 0 X 104,0.176987


In [ ]:
lgb_azer.rename(columns = {'preds_test':'target'},inplace=True)
lgb_azer.sort_values(by='CID X LOC_NUM X VENDOR',inplace=True)
lgb_azer.reset_index(drop=True,inplace=True)
lgb_azer.head(1)

,CID X LOC_NUM X VENDOR,target
0,000IPH5 X 0 X 104,0.235992


In [ ]:
lgb_nacir.sort_values(by='CID X LOC_NUM X VENDOR',inplace=True)
lgb_nacir.reset_index(drop=True,inplace=True)
lgb_nacir.head(1)

,CID X LOC_NUM X VENDOR,target
0,000IPH5 X 0 X 104,0.240278


In [ ]:
xgb_azer.sort_values(by='CID X LOC_NUM X VENDOR',inplace=True)
xgb_azer.reset_index(drop=True,inplace=True)
xgb_azer.head(1)

,CID X LOC_NUM X VENDOR,target
0,000IPH5 X 0 X 104,0.317326


In [ ]:
blend = pd.DataFrame()
blend['CID X LOC_NUM X VENDOR'] = lgb_ahmed['CID X LOC_NUM X VENDOR']
blend['target'] = lgb_nacir.target*0.47 + lgb_azer.target*0.29 + lgb_ahmed.target*0.14 + xgb_azer.target*0.1
blend.head(1)

,CID X LOC_NUM X VENDOR,target
0,000IPH5 X 0 X 104,0.237879


In [ ]:
blend['target'] = (blend['target']>0.475).astype('int')
blend['target'].value_counts()

0    1638732
1      33268
Name: target, dtype: int64

# **FIRST BEST BLEND  + SECOND BEST BLEND**

In [8]:
blend = pd.read_csv('/content/drive/My Drive/AKEED_SOLUTION_Predictions/CheckBlenAhmed.csv')

In [9]:
sub = sub.sort_values('CID X LOC_NUM X VENDOR',ascending=False)
blend = blend.sort_values('CID X LOC_NUM X VENDOR',ascending=False)

In [10]:
def binary_blend(sub1,sub2) :
  final_blend = []
  for i,j in zip(sub1,sub2) :
    x= i&j
    final_blend.append(x)

  return final_blend

In [11]:
final_blend = blend.copy()
final_blend['target']= binary_blend(list(blend['target'].values),
                            list(sub['target'].values),)

In [12]:
final_blend['target'].value_counts()

0    1641641
1      30359
Name: target, dtype: int64

In [13]:
final_blend.to_csv('BestScore.csv',index=False)